In [1]:
import Pkg
Pkg.add("GLPK")

    Updating registry at `C:\Users\calve\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\calve\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\calve\.julia\environments\v1.9\Manifest.toml`


In [2]:
using JuMP
using GLPK

function solve_delta_tsp(distances)
    n = size(distances, 1)
    model = Model(GLPK.Optimizer)
    @variable(model, x[1:n, 1:n], Bin)  # Variable binaire pour indiquer si le trajet passe par l'arc (i, j)
    @variable(model, u[1:n])  # Variables de position
    @objective(model, Min, sum(distances[i, j] * x[i, j] for i in 1:n, j in 1:n)) 
    # Contrainte: on ne bloque pas sur la même ville
    @constraint(model, [i in 1:n,], x[i, i] == 0)
    # Contrainte: chaque ville est visitée exactement une fois
    @constraint(model, [i in 1:n], sum(x[i, j] for j in 1:n) == 1)
    @constraint(model, [j in 1:n], sum(x[i, j] for i in 1:n) == 1)
    # Contrainte pour éviter les sous-tours
    @constraint(model, [i=1:n, j=2:n], u[i] - u[j] + n * x[i, j] <= n - 1)
    # Résolution
    optimize!(model)
    if termination_status(model) == MOI.OPTIMAL
        println("Optimal solution found!")
        selected_arcs = value.(x)
        for i in 1:n
            for j in 1:n
                if selected_arcs[i, j] > 0.5
                    println("On voyage de ", i, " vers ", j)
                end
            end
        end
    else
        println("No solution found!")
    end
    println("distance total parcouru ",value.(sum(distances[i, j] * x[i, j] for i in 1:n, j in 1:n)))
end

# Génération aléatoire des coordonnées des villes
using Random
Random.seed!(123)  # Pour la reproductibilité
function calculate_distances(coordonnees)
    nbv = length(coordonnees)
    dist = zeros(nbv, nbv)
    for i in 1:nbv, j in 1:nbv
        dist[i, j] = sqrt((coordonnees[i][1] - coordonnees[j][1])^2 + (coordonnees[i][2] - coordonnees[j][2])^2)
    end
    return dist
end


max_coord = 100  # Plage des coordonnées
N = 16
coordonnees = [(rand(1:max_coord), rand(1:max_coord)) for _ in 1:N]
d = zeros(N, N)
#println(d)
d = calculate_distances(coordonnees)
@time solve_delta_tsp(d)


Optimal solution found!
On voyage de 1 vers 15
On voyage de 2 vers 12
On voyage de 3 vers 5
On voyage de 4 vers 7
On voyage de 5 vers 10
On voyage de 6 vers 11
On voyage de 7 vers 14
On voyage de 8 vers 13
On voyage de 9 vers 1
On voyage de 10 vers 2
On voyage de 11 vers 4
On voyage de 12 vers 9
On voyage de 13 vers 3
On voyage de 14 vers 16
On voyage de 15 vers 6
On voyage de 16 vers 8
distance total parcouru 321.74481188754646
  4.067043 seconds (4.18 M allocations: 285.535 MiB, 5.08% gc time, 97.47% compilation time)


In [3]:
N = 12

d = [0 1 1 100 100 100;
     1 0 1 100 100 100;
    1 1 0 100 100 100;
100 100 100 0 1 1;
100 100 100 1 0 1;
100 100 100 1 1 0]
@time solve_delta_tsp(d)


Optimal solution found!
On voyage de 1 vers 3
On voyage de 2 vers 4
On voyage de 3 vers 2
On voyage de 4 vers 5
On voyage de 5 vers 6
On voyage de 6 vers 1
distance total parcouru 204.0
  0.168370 seconds (160.54 k allocations: 10.845 MiB, 96.66% compilation time)


In [18]:
function placer(N, distance, solution, score_actuel, meilleure_solution, meilleur_score)
    
    if length(solution)==N
        if score_actuel[1]+distance[solution[1], solution[N]]<meilleur_score[1]
            meilleure_solution[1] = solution
            meilleur_score[1] = score_actuel[1] + distance[solution[1], solution[N]]
        end
    else
        dernier_sommet = solution[length(solution)]
        for i in 1:N
            if (!(i in solution)) && score_actuel[1]+distance[i, dernier_sommet]<meilleur_score[1]
                push!(solution, i)
                placer(N, distance, solution, [score_actuel[1]+distance[i, dernier_sommet]], meilleure_solution, meilleur_score)
                pop!(solution)
            end
        end  
    end
end

function backtracking(N, distance)
        meilleure_solution = [[]]
        placer(N, distance, [1], [0.0], meilleure_solution, [10000000.0])
        println(meilleure_solution)
end
            

N = 14
somme=0
for i in 1:10
    coordonnees = [(rand(1:100), rand(1:100)) for _ in 1:N]
    d = calculate_distances(coordonnees)
    t1 = time()
    @time solve_delta_tsp(d)
    elapsed_time = time()-t1
    println(elapsed_time)
    somme+=elapsed_time
end
println(somme/10)
        

Optimal solution found!
On voyage de 1 vers 8
On voyage de 2 vers 12
On voyage de 3 vers 11
On voyage de 4 vers 6
On voyage de 5 vers 1
On voyage de 6 vers 9
On voyage de 7 vers 2
On voyage de 8 vers 4
On voyage de 9 vers 13
On voyage de 10 vers 14
On voyage de 11 vers 7
On voyage de 12 vers 5
On voyage de 13 vers 10
On voyage de 14 vers 3
distance total parcouru 309.20615229690935
  1.453448 seconds (52.46 k allocations: 3.786 MiB)
1.4539999961853027
Optimal solution found!
On voyage de 1 vers 11
On voyage de 2 vers 7
On voyage de 3 vers 2
On voyage de 4 vers 10
On voyage de 5 vers 6
On voyage de 6 vers 14
On voyage de 7 vers 8
On voyage de 8 vers 1
On voyage de 9 vers 3
On voyage de 10 vers 13
On voyage de 11 vers 4
On voyage de 12 vers 5
On voyage de 13 vers 12
On voyage de 14 vers 9
distance total parcouru 296.7490727739286
  1.386460 seconds (43.63 k allocations: 3.661 MiB)
1.386000156402588
Optimal solution found!
On voyage de 1 vers 10
On voyage de 2 vers 4
On voyage de 3 vers 1

In [42]:
using DelimitedFiles

M = readdlm("main1/instances.txt")
m, N = size(M)
k = Integer(m/N)
fichier_entree = open("main1/datas_exacte.csv", "r")
lignes = readlines(fichier_entree)
close(fichier_entree)


for i in 1:k
    d=M[(i-1)*N+1:i*N,:]
    
    t1 = time()
    solve_delta_tsp(d)
    elapsed_time = time()-t1
    
    lignes[i+2]*=string(round(elapsed_time, digits=3))
    println(lignes[i+2])
end

fichier_sortie = open("main1/datas_exacte.csv", "w")
write(fichier_sortie, join(lignes, "\n"))
close(fichier_sortie)



Optimal solution found!
On voyage de 1 vers 2
On voyage de 2 vers 9
On voyage de 3 vers 5
On voyage de 4 vers 3
On voyage de 5 vers 7
On voyage de 6 vers 4
On voyage de 7 vers 10
On voyage de 8 vers 6
On voyage de 9 vers 8
On voyage de 10 vers 1
distance total parcouru 362.0
1.917,0.144,21,0.382,109,0.001,0.031
Optimal solution found!
On voyage de 1 vers 5
On voyage de 2 vers 10
On voyage de 3 vers 9
On voyage de 4 vers 2
On voyage de 5 vers 7
On voyage de 6 vers 8
On voyage de 7 vers 4
On voyage de 8 vers 1
On voyage de 9 vers 6
On voyage de 10 vers 3
distance total parcouru 410.0
2.925,2.675,309,0.768,199,0,0.033
Optimal solution found!
On voyage de 1 vers 8
On voyage de 2 vers 10
On voyage de 3 vers 5
On voyage de 4 vers 2
On voyage de 5 vers 1
On voyage de 6 vers 4
On voyage de 7 vers 6
On voyage de 8 vers 7
On voyage de 9 vers 3
On voyage de 10 vers 9
distance total parcouru 354.0
4.679,31.514,1831,1.945,952,0,0.048
Optimal solution found!
On voyage de 1 vers 3
On voyage de 2 vers